<h2>3D stack - Batch processing - Marker+ based on average intensity</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) and using the parameters tested in <code>000_SP_Avg_Intensity</code> allows you to:

1. Read previously defined ROIs, if not present, full image is analyzed.
2. Read previously predicted nuclei labels, if not present, generates them.
3. Extract numbers of cells positive for all marker based on signal average intensity within the nuclear or cytoplasmic compartments (using a user-defined min-max range).
4. Extract and save per label per ROI per marker data in a .csv file (filename_per_label_avg_int_.csv).
5. Extract and save number and % of positive cells in a .csv file (BP_marker_+_label_avg_int.csv).

In [1]:
from pathlib import Path
import tifffile
import napari
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from skimage.measure import regionprops_table
from stardist.models import StarDist3D
from utils_stardist import get_gpu_details, list_images, read_image, maximum_intensity_projection, extract_nuclei_stack, simulate_cytoplasm_chunked_3d, simulate_cell_chunked_3d, segment_nuclei_3d

get_gpu_details()

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


<h3>Define the directory for your images (.nd2 or .czi files) and cell marker info</h3>

In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# cellular locations can be "nucleus", "cytoplasm" or "cell" (cell being the sum volume of nucleus and cytoplasm)
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR1.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M7_S11_TR1.czi',
 '..\\raw_data\\t

<h3>Define you batch analysis parameters</h3>

If you have generated nuclei predictions already, make sure to input the same <code>slicing factor</code> you used when generating nuclei predictions. 

If you have not generated nuclei predictions before, input <code>nuclei_channel</code>, <code>n_tiles</code>, <code>segmentation_type</code> and <code>model_name</code> values.

In [3]:
# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# The n_tiles parameter defines the number of tiles the input volume/image will be divided into along each dimension (z, y, x) during prediction. 
# This is useful for processing large images that may not fit into memory at once.
# While tiling can handle memory limitations, chopping the image into smaller chunks increases
# the processing time for stitching the predictions back together. 
# Use n_tiles=(1, 1, 1) if the input volume fits in memory without tiling to minimize processing overhead.
n_tiles=(1,4,4)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
model_name = "MEC0.1"

# Model loading 
model = StarDist3D(None, name=model_name, basedir='stardist_models')

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.583933, nms_thresh=0.3.


Define in <code>min_max_per_marker</code> the <code>marker</code> you want to use to define your cell populations of interest, the <code>min_max</code> range of avg_int and the <code>population</code> name.

In [4]:
 # Define the min_max average intensity parameters to select your populations of interest
# You have the possibility to define populations for the same marker (i.e. neun high and neun low)
# max_values are set to 255 since the test input images are 8-bit, higher bit depths can result in higher max avg_int values

min_max_per_marker = [{"marker": "ki67", "min_max": (110,255), "population":"ki67"},
                      {"marker": "neun", "min_max": (20,80), "population":"neun_low"},
                      {"marker": "neun", "min_max": (80,255), "population":"neun_high"},
                      {"marker": "calbindin", "min_max": (25,255), "population":"calbindin"},]

<h3>Run Batch Analysis</h3>

In [5]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds" / segmentation_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

for image in tqdm(images):

    # Read image, apply slicing if needed and return filename and img as a np array
    img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

    # Generate maximum intensity projection 
    img_mip = maximum_intensity_projection(img)

    # Initialize an empty list to hold the extracted dataframes on a per ROI basis
    per_roi_props = []

    for roi_name in roi_names:

        print(f"\nAnalyzing ROI: {roi_name}")

        # Initialize an empty list to hold the extracted dataframes on a per channel basis
        props_list = []

        # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
        try:
            # Read previously defined ROIs
            user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

        except FileNotFoundError:
            # Extract the xy dimensions of the input image 
            img_shape = img_mip.shape
            img_xy_dims = img_shape[-2:]

            # Create a label covering the entire image
            user_roi = np.ones(img_xy_dims).astype(np.uint8)

        # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
        try:
            # Read the nuclei predictions per ROI
            nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")
            print(f"Pre-computed nuclei labels found for {filename}")

        except FileNotFoundError:

            print(f"Generating nuclei labels for {filename}")
            
            # Slice the nuclei stack
            nuclei_img = extract_nuclei_stack(img, nuclei_channel)

            # We will create a mask where roi is greater than or equal to 1
            mask = (user_roi >= 1).astype(np.uint8)

            # 3D segmentation logic, extend 2D mask across the entire stack volume
            if segmentation_type == "3D":
                # Extract the number of z-slices to extend the mask
                slice_nr = img.shape[1]
                # Extend the mask across the entire volume
                mask = np.tile(mask, (slice_nr, 1, 1))
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)
            else:
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)

            # Segment nuclei and return labels
            nuclei_labels = segment_nuclei_3d(masked_nuclei_img, model, n_tiles)

            # Save nuclei labels as .tiff files to reuse them later
            try:
                os.makedirs(nuclei_preds_path / roi_name, exist_ok=True)
            except Exception as e:
                print(f"Error creating directory {nuclei_preds_path / roi_name}: {e}")

            # Construct path to store
            path_to_store = nuclei_preds_path / roi_name / f"{filename}.tiff"
            print(f"Saving nuclei labels to {path_to_store}")
            try:
                tifffile.imwrite(path_to_store, nuclei_labels)
            except Exception as e:
                print(f"Error saving file {path_to_store}: {e}")

        # Create a dictionary containing all image descriptors
        descriptor_dict = {
                    "filename": filename,
                    "ROI": roi_name,
                    }

        # Loop through each channel and extract the average intensity within either nuclei or cytoplasmic regions
        for tuple in markers:

            channel_name = tuple[0]
            ch_nr = tuple[1]
            location = tuple[2]

            print(f"Analyzing channel: {channel_name}")

            if location == "cytoplasm":

                print(f"Generating cytoplasm labels for: {channel_name}")

                # Simulate a cytoplasm by dilating the nuclei and substracting the nuclei mask afterwards
                cytoplasm_labels = simulate_cytoplasm_chunked_3d(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

                # Extract intensity information from each marker channel
                props = regionprops_table(label_image=cytoplasm_labels,
                                    intensity_image=img[ch_nr],
                                    properties=["label", "intensity_mean"])
                
            elif location == "cell":

                print(f"Generating cell labels for: {channel_name}")

                # Simulate a cell by dilating the nuclei
                cell_labels = simulate_cell_chunked_3d(nuclei_labels, dilation_radius = 2, erosion_radius = 0)

                # Extract intensity information from each marker channel
                props = regionprops_table(label_image=cell_labels,
                                    intensity_image=img[ch_nr],
                                    properties=["label", "intensity_mean"])
            
            elif location == "nucleus":

                # Extract intensity information from each marker channel
                props = regionprops_table(label_image=nuclei_labels,
                                    intensity_image=img[ch_nr],
                                    properties=["label", "intensity_mean"])
            

            # Convert to dataframe
            props_df = pd.DataFrame(props)

            # Rename intensity_mean column to indicate the specific image
            props_df.rename(columns={"intensity_mean": f"{location}_{channel_name}_avg_int"}, inplace=True)

            # Append each props_df to props_list
            props_list.append(props_df)

            # Initialize the df with the first df in the list
            props_df = props_list[0]
            # Start looping from the second df in the list
            for df in props_list[1:]:
                props_df = props_df.merge(df, on="label")

        # Add each key-value pair from descriptor_dict to props_df at the specified position
        insertion_position = 0    
        for key, value in descriptor_dict.items():
            props_df.insert(insertion_position, key, value)
            insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

        # Append each props_df to props_list
        per_roi_props.append(props_df)

    final_df = pd.concat(per_roi_props, ignore_index=True)

    # Create a 'results' folder in the root directory
    results_folder = Path("results") / experiment_id / segmentation_type / model_name

    try:
        os.makedirs(results_folder)
        print(f"'{results_folder}' folder created successfully.")
    except FileExistsError:
        print(f"'{results_folder}' folder already exists.")

    # Save the df containing per_label results into a CSV file
    final_df.to_csv(results_folder / f'{filename}_per_label_avg_int.csv')

    # Select all column names in 'final_df' that contain the substring 'avg_int'
    avg_int_columns = [col for col in final_df.columns if 'avg_int' in col]

    # Create an empty list to store all stats extracted from each image
    stats = []

    for marker_analysis in min_max_per_marker:

        marker = marker_analysis["marker"]
        min_max_avg_int = marker_analysis["min_max"]
        population = marker_analysis["population"]

        # Retrieve the column name from which the avg_int values should be read
        for column in avg_int_columns:
            if marker in column:
                column_name = column

        for roi_name in roi_names:

            # Initialize an empty list to hold the extracted dataframes on a per channel basis
            props_list = []

            # Retrieve the first and second values (channel and location) of the corresponding tuple in markers
            for item in markers:
                if item[0] == marker:
                    channel = item[1]
                    location = item[2]
                    break  # Stop searching once the marker is found

            # Read the nuclei predictions per ROI
            nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")

            # Filter rows in final_df where ROI matches roi_name and column_name is within min < avg_int <= max values.
            filtered_df = final_df[(final_df["ROI"] == roi_name) & (final_df[column_name] > min_max_avg_int[0]) & (final_df[column_name] <= min_max_avg_int[1])]

            # Get the values of the 'label' column in filtered_df as a list
            label_values = filtered_df["label"].tolist()

            # Create a boolean mask where each element is True if the corresponding value in 'nuclei_labels' 
            # is found in 'label_values', and False otherwise
            mask = np.isin(nuclei_labels, label_values)

            # Use the mask to set values in 'nuclei_labels' that are not in 'label_values' to 0,
            # creating a new array 'filtered_labels' with only the specified values retained
            filtered_labels = np.where(mask, nuclei_labels, 0)

            # Extract your information of interest
            total_nuclei = len(np.unique(nuclei_labels)) - 1
            marker_pos_nuclei = len(np.unique(filtered_labels)) - 1

            # Calculate "%_marker+_cells" and avoid division by zero errors
            try:
                perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
            except ZeroDivisionError:
                perc_marker_pos_cells = 0

            # Create a dictionary containing all extracted info per masked image
            stats_dict = {
                        "filename": filename,
                        "ROI": roi_name,
                        "based_on": column_name,
                        "marker": marker,
                        "population":population,
                        "marker_ch": channel,
                        "location": location,
                        "min_max_avg_int": min_max_avg_int,
                        "total_nuclei": total_nuclei,
                        "marker+_nuclei": marker_pos_nuclei,
                        "%_marker+_cells": perc_marker_pos_cells,
                        "slicing_factor_xy": slicing_factor_xy,
                        "slicing_factor_z": slicing_factor_z
                        }
            
            # Append the current data point to the stats_list
            stats.append(stats_dict)  

    # Create the necessary folder structure if it does not exist
    try:
        os.makedirs(str(results_folder))
        print(f"Output folder created: {results_folder}")
    except FileExistsError:
        print(f"Output folder already exists: {results_folder}")

    # Transform into a dataframe to store it as .csv later
    df = pd.DataFrame(stats)

    # Define the .csv path
    csv_path = results_folder / f"BP_marker_+_label_avg_int.csv"

    # Append to the .csv with new data points each round
    df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df 

The following regions of interest will be analyzed: ['CA', 'DG']


  0%|          | 0/23 [00:00<?, ?it/s]



Image analyzed: HI1_CONTRA_M8_S6_TR1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)

Analyzing ROI: CA
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin

Analyzing ROI: DG
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin
'results\test_data\3D\MEC0.1' folder already exists.


  4%|▍         | 1/23 [01:20<29:20, 80.03s/it]

Output folder already exists: results\test_data\3D\MEC0.1


Image analyzed: HI1_CONTRA_M8_S6_TR2
Original Array shape: (4, 13, 3802, 2882)
Compressed Array shape: (4, 13, 3802, 2882)

Analyzing ROI: CA
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR2
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin

Analyzing ROI: DG
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR2
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin
'results\test_data\3D\MEC0.1' folder already exists.


...

 96%|█████████▌| 22/23 [18:16<00:58, 58.70s/it]

Output folder already exists: results\test_data\3D\MEC0.1


Image analyzed: SHAM1_IPSI_M7_S11_TR2
Original Array shape: (4, 15, 2883, 2889)
Compressed Array shape: (4, 15, 2883, 2889)

Analyzing ROI: CA
Pre-computed nuclei labels found for SHAM1_IPSI_M7_S11_TR2
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin

Analyzing ROI: DG
Pre-computed nuclei labels found for SHAM1_IPSI_M7_S11_TR2
Analyzing channel: ki67
Analyzing channel: neun
Analyzing channel: calbindin
Generating cytoplasm labels for: calbindin
'results\test_data\3D\MEC0.1' folder already exists.


100%|██████████| 23/23 [19:03<00:00, 49.71s/it]

Output folder already exists: results\test_data\3D\MEC0.1


,filename,ROI,based_on,marker,population,marker_ch,location,min_max_avg_int,total_nuclei,marker+_nuclei,%_marker+_cells,slicing_factor
0,HI1_CONTRA_M8_S6_TR1,CA,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(110, 255)",1518,30,1.976285,NaN
1,HI1_CONTRA_M8_S6_TR1,DG,nucleus_ki67_avg_int,ki67,ki67,0,nucleus,"(110, 255)",2113,114,5.395173,NaN
2,HI1_CONTRA_M8_S6_TR1,CA,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(20, 80)",1518,401,26.416337,NaN
3,HI1_CONTRA_M8_S6_TR1,DG,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(20, 80)",2113,1378,65.215334,NaN
4,HI1_CONTRA_M8_S6_TR1,CA,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(80, 255)",1518,885,58.300395,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
179,SHAM1_IPSI_M7_S11_TR2,DG,nucleus_neun_avg_int,neun,neun_low,1,nucleus,"(20, 80)",1343,542,40.357409,NaN
180,SHAM1_IPSI_M7_S11_TR2,CA,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(80, 255)",832,643,77.283654,NaN
181,SHAM1_IPSI_M7_S11_TR2,DG,nucleus_neun_avg_int,neun,neun_high,1,nucleus,"(80, 255)",1343,724,53.909159,NaN
182,SHAM1_IPSI_M7_S11_TR2,CA,cytoplasm_calbindin_avg_int,calbindin,calbindin,2,cytoplasm,"(25, 255)",832,702,84.375000,NaN
